In [1]:
import os

In [2]:
%pwd

'/Users/aadya/Documents/GitHub/NER-Pipelines/research'

In [3]:
os.chdir("../")

In [4]:
os.getcwd()

'/Users/aadya/Documents/GitHub/NER-Pipelines'

In [5]:
import sys
# sys.path is a list of absolute path strings
sys.path.insert(0, os.path.abspath('/Users/aadya/Documents/GitHub/NER-Pipelines/src/'))
sys.path.append('/Users/aadya/Documents/GitHub/NER-Pipelines')
sys.path

['/Users/aadya/Documents/GitHub/NER-Pipelines/src',
 '/Users/aadya/Documents/GitHub/NER-Pipelines/research',
 '/Users/aadya/anaconda3/envs/ner-pipeline/lib/python311.zip',
 '/Users/aadya/anaconda3/envs/ner-pipeline/lib/python3.11',
 '/Users/aadya/anaconda3/envs/ner-pipeline/lib/python3.11/lib-dynload',
 '',
 '/Users/aadya/anaconda3/envs/ner-pipeline/lib/python3.11/site-packages',
 '/Users/aadya/Documents/GitHub/NER-Pipelines']

In [6]:
modulenames = set(sys.modules) & set(globals())
allmodules = [sys.modules[name] for name in modulenames]
allmodules

[<module 'os' (frozen)>, <module 'sys' (built-in)>]

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [17]:
!pip install --upgrade ensure

In [8]:
from ner_pipelines.constants import *
from ner_pipelines.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [13]:
import os
import urllib.request as request
import zipfile
from ner_pipelines import logger

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info(self.config.source_URL)
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-28 23:39:23,571: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-28 23:39:23,573: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-28 23:39:23,574: INFO: common: created directory at: artifacts]
[2023-10-28 23:39:23,575: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-28 23:39:23,575: INFO: 3726761732: https://github.com/pritam1984314/NER-Pipelines/raw/main/ner_data.zip]
[2023-10-28 23:39:26,877: INFO: 3726761732: artifacts/data_ingestion/net_data download! with following info: 
Connection: close
Content-Length: 3239199
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d7838bac782a550a59378c731e7338479d6549f481796ad0c21e4dbb3d6b67df"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 759C:815D:4728CE:58970C:653D4E50
Acce